# LLM graph iteration demo

Anton Antonov   
January 2026

---

## Introduction

This notebook shows how to do iterative refinements of results agent results using a iterative evaluation of an (acyclic) LLM-graph.

---

## Setup

In [15]:
use LLM::Graph;
use WWW::Ollama;

In [16]:
sink my $conf-ollama = LLM::Functions::Configuration.new(
        name => 'ollama',
        api-key => Whatever,
        api-user-id => 'user:' ~ ((10 ** 11 + 1) .. 10 ** 12).pick,
        module => 'WWW::Ollama',
        base-url => ollama-base-url,
        model => 'gemma3:4b',
        function => &ollama-chat-completion,
        embedding-model => 'nomic-embed-text',
        embedding-function => &ollama-embedding,
        temperature => 0.4,
        max-tokens => 8192,
        total-probability-cutoff => 0,
        prompts => Empty,
        prompt-delimiter => ' ',
        examples => Empty,
        stop-tokens => Empty,
        argument-renames => %(),
        format => 'values');

In [ ]:
sink my $conf41 = llm-configuration('ChatGPT', model => 'gpt-4.1', temperature => 0.55, max-tokens => 8192);

In [18]:
sink my $conf = $conf41-mini;

----

## Graph nodes

The cycle is: generate -> critique -> revise -> critique -> ... and stops when the critique says "acceptable" or max iterations is reached.

### Generation rules

In [19]:
sink my %generation-rules =

    generate => sub ($topic) { "Generate a 200 words thesis text for the topic $topic." ~ llm-prompt('NothingElse')('thesis text without comments') }
;

### Critique LLM-function

In [20]:
my &cFunc = llm-function( -> $text {
    [
        "Write a VERY SHORT critique of the thesis below.",
        "Give your verdict to be one of 'BAD', 'ACCEPTABLE', 'GOOD'.",
        "If you verdict is 'BAD' or 'ACCEPTABLE' give improvement directions only.",
        "THESIS:\n", 
        $text ~~ Str:D ?? $text !! $text<revision>
    ].join("\n")
}, e => $conf);

LLM::Function(-> **@args, *%args { #`(Block|6340274840720) ... }, 'chatgpt')

Critique prompt variant:

```raku
    [
        "Write a VERY SHORT critique of the thesis below.",
        "Evaluate with questions like:",
        "1) Is it too short? 2) Does it have at least two sections? 3) Is it easy to read?",
        "Give your verdict to be one of 'BAD', 'ACCEPTABLE', 'GOOD'.\n",
        "THESIS:\n", 
    ]
```    

### Revision LLM-function

In [21]:
my &rFunc = llm-function(-> $critique-form {
    [
        "Here is a thesis:",
        "\nTHESIS:\n", 
        $critique-form<revision>,
        "\nEND-THESIS\n",
        "And here is a critique of it:",
        "CRITIQUE:\n",
        $critique-form<critique>,
        "END-CRITIQUE\n",
        "Improve the thesis text based on the critique.",
        "Do not comment, annotate, or give options."
    ].join("\n")
}, e => $conf);

LLM::Function(-> **@args, *%args { #`(Block|6340274875992) ... }, 'chatgpt')

### Refinement rules

In [22]:
my %empty-form = critique => '', critique-iteration => 0, revision => '', revision-iteration => 0;

sink my %revision-rules =

    decide => { 
        eval-function => sub ($text) { 
            $text ~~ Str:D || !($text ~~ Map:D && $text<critique>.contains('GOOD'))
        } 
    },

    critique => {
        eval-function => sub ($text) { &cFunc($text) },
        
        test-function => sub ($decide) { $decide.raku.lc.contains('true') }
    },

    critique-form => {
        eval-function => sub ($text, $critique) {
            my $form = $text ~~ Map:D ?? $text.clone !! merge-hash(%empty-form , {revision => $text});
            $form<critique> = $critique;
            $form<critique-iteration> += 1;
            return $form;
        },
        
        test-function => sub ($critique) { $critique.defined }
    },

    revise => {
        eval-function => sub ($critique-form) { &rFunc($critique-form) },
        
        test-function => sub ($critique-form) { $critique-form.defined },
    },

    revise-form => {
        eval-function => sub ($critique-form, $revise) {
            my %form = $critique-form.clone; 
            %form<revision> = $revise;
            %form<revision-iteration> += 1;
            return %form;
        },
        
        test-function => sub ($revise) { $revise.defined }
    },

    finalize => {
        eval-function => sub ($text, $revise-form) { $revise-form.defined ?? $revise-form !! $text}
    }  
;

---


## Build the graphs

In [23]:
my $g1 = LLM::Graph.new(%generation-rules, e => $conf):!async;
my $g2 = LLM::Graph.new(%revision-rules, e => $conf):!async;

(:$g1, :$g2)

(g1 => LLM::Graph(size => 1, nodes => generate) g2 => LLM::Graph(size => 6, nodes => critique, critique-form, decide, finalize, revise, revise-form))

---

## Run and inspect state transitions

In [24]:
$g1.eval({ topic => "why cyclic graphs help with iterative writing" })

LLM::Graph(size => 1, nodes => generate)

In [25]:
my $text = $g1.nodes<generate><result>

Cyclic graphs significantly enhance iterative writing by providing a structured framework that mirrors the recursive nature of the writing process. Unlike linear models, cyclic graphs allow writers to visualize and manage the continuous feedback loops inherent in drafting, revising, and refining text. This representation facilitates a clear understanding of how ideas evolve through repeated cycles of evaluation and modification. By mapping out these cycles, writers can identify points where revisions are necessary, thereby promoting more efficient and focused iterations. Additionally, cyclic graphs support the integration of diverse inputs, such as peer feedback and self-assessment, at multiple stages, reinforcing the dynamic and non-linear progression of writing. This approach aligns with cognitive theories that emphasize reflection and revision as critical components of effective writing. Furthermore, cyclic graphs help in organizing complex writing tasks by breaking them into manage

In [27]:
$text.words.elems

199

In [28]:
$g2.eval({ :$text })

LLM::Graph(size => 6, nodes => critique, critique-form, decide, finalize, revise, revise-form)

In [29]:
#% markdown
$g2.nodes<finalize><result>

{critique => Verdict: ACCEPTABLE

Improvement directions:
- Include empirical evidence or case studies to support claims about the effectiveness of cyclic graphs in iterative writing.
- Clarify how cyclic graphs compare quantitatively or qualitatively to other visualization methods.
- Address potential limitations or challenges in applying cyclic graphs to diverse writing contexts.
- Simplify some complex sentences to improve readability and impact., critique-iteration => 1, revision => Cyclic graphs significantly enhance iterative writing by providing a structured framework that mirrors the recursive nature of the writing process. Unlike linear models, cyclic graphs allow writers to visualize and manage the continuous feedback loops inherent in drafting, revising, and refining text. This representation facilitates a clear understanding of how ideas evolve through repeated cycles of evaluation and modification. For example, studies have shown that writers using cyclic visualizations complete revisions more efficiently and produce higher-quality drafts compared to those relying on linear outlines. By mapping out these cycles, writers can identify points where revisions are necessary, promoting more focused and effective iterations.

Additionally, cyclic graphs support the integration of diverse inputs, such as peer feedback and self-assessment, at multiple stages, reinforcing the dynamic and non-linear progression of writing. Compared to other visualization methods, such as hierarchical trees or flowcharts, cyclic graphs uniquely capture the iterative loops that characterize the writing process, offering a more accurate and flexible representation. However, applying cyclic graphs may present challenges in certain contexts, such as brief or highly structured writing tasks, where the complexity of the graph might overwhelm rather than assist the writer.

This approach aligns with cognitive theories that emphasize reflection and revision as critical components of effective writing. Furthermore, cyclic graphs help organize complex writing tasks by breaking them into manageable phases connected through iterative loops. This organization reduces cognitive overload and enhances creativity by allowing writers to focus on one cycle at a time. For instance, case studies in academic writing have demonstrated that cyclic graph frameworks lead to deeper engagement and more thoughtful revisions.

In essence, the use of cyclic graphs transforms iterative writing from a chaotic process into a systematic, transparent, and adaptable workflow. Therefore, cyclic graphs not only improve the quality of the final text but also foster a deeper engagement with the writing process, ultimately leading to more polished and thoughtful compositions., revision-iteration => 1}

In [30]:
$g1.clear;
$g2.clear;

$g1.eval({ topic => "Why cyclic graphs help with iterative writing?" });
my $text = $g1.nodes<generate><result>;

for (^4) -> $iter {
  say '=' x 10, $iter, '=' x 10;
  $g2.eval({:$text});  
  my $revision = $g2.nodes<finalize><result>;
  last if $revision eq $text;
  $text = $revision;
  say 'critique => ', $g2.nodes<critique><result>;
  say "-=" x 10, "-";
  say to-json($text);
}

==========0==========
critique => Verdict: ACCEPTABLE

Improvement directions:
- Provide concrete examples or case studies demonstrating how cyclic graphs improve writing processes.
- Clarify potential limitations or challenges in implementing cyclic graphs in writing workflows.
- Include references to existing research or tools that utilize cyclic graphs to strengthen the argument.
- Make the language more concise to enhance readability and impact.
-=-=-=-=-=-=-=-=-=-=-
{
  "critique-iteration": 1,
  "critique": "Verdict: ACCEPTABLE\n\nImprovement directions:\n- Provide concrete examples or case studies demonstrating how cyclic graphs improve writing processes.\n- Clarify potential limitations or challenges in implementing cyclic graphs in writing workflows.\n- Include references to existing research or tools that utilize cyclic graphs to strengthen the argument.\n- Make the language more concise to enhance readability and impact.",
  "revision-iteration": 1,
  "revision": "Cyclic gra

### Final output

In [31]:
#% markdown
$text<revision>

Cyclic graphs enhance iterative writing by offering a structured yet flexible framework that reflects the recursive nature of the writing process. A cyclic graph is a visual representation consisting of nodes and directed edges that form loops, allowing for repeated traversal of certain stages or sections. Unlike linear models, cyclic graphs create continuous feedback loops, enabling writers to revisit and refine earlier sections based on new insights or external input. This cyclical approach aligns with the repetitive drafting, revising, and editing inherent in writing.

Empirical studies have demonstrated the effectiveness of cyclic graphs in academic collaborations. For example, a study involving four academic writing teams using cyclic graphs to map revision cycles reported a 25% improvement in tracking changes and dependencies, which led to more coherent and consistent documents. In a controlled experiment with 60 participants divided into two groups, those employing cyclic graphs reduced revision time by an average of 15% compared to those using traditional linear methods, with statistical significance (p < 0.05). By visually representing iterations, cyclic graphs clarify the flow of ideas and dependencies, aiding writers in maintaining coherence and consistency throughout the drafting process. Additionally, they support collaborative writing by mapping contributions from multiple authors, thereby improving communication and coordination.

Tools such as the revision graph feature in GitBook exemplify how cyclic graphs can manage complex writing tasks by breaking them into manageable cycles and tracking progress efficiently. To evaluate the effectiveness of cyclic graphs, mixed-method approaches combining time-tracking metrics, revision quality assessments, and user satisfaction surveys have been employed, providing a comprehensive understanding of their impact on writing workflows. However, implementing cyclic graphs may pose challenges, including a learning curve for writers unfamiliar with graph-based tools and the potential to overcomplicate simpler writing projects. For instance, in brief or highly structured documents such as executive summaries or standardized reports, the added complexity of cyclic graphs may not yield significant benefits. Furthermore, usability concerns arise when the graphical interface becomes too complex or when writers prefer more traditional linear workflows. In contexts where writing tasks are straightforward or time-constrained, cyclic graphs may offer limited advantages.

Comparative analyses indicate that cyclic graphs outperform some traditional iterative writing methods by providing clearer visualization of dependencies and revision history, although quantitative data on efficiency gains remain limited and warrant further research. Alternative models, such as linear checklists or hierarchical outlines, offer simplicity but lack the dynamic feedback loops intrinsic to cyclic graphs, which better mirror the nonlinear nature of writing. This study aims to investigate how cyclic graphs influence iterative writing efficiency and collaboration quality compared to conventional tools, employing controlled experiments, user experience evaluations, and document quality metrics to quantify their impact and delineate context-specific applicability.

Despite these limitations, cyclic graphs transform the nonlinear writing process into a tangible, navigable system that promotes both efficiency and creativity. Consequently, they serve as valuable tools in both individual and collaborative iterative writing, fostering continuous improvement and deeper engagement with the text. By explicitly addressing their scope, benefits, challenges, and employing rigorous methodologies, this research contributes to a nuanced understanding of how cyclic graphs can be effectively integrated into diverse writing workflows.

In [32]:
#% markdown
$g2.nodes<critique-form>

{eval-function => sub { }, input => [critique text], result => {critique => Verdict: GOOD

The thesis presents a clear, well-structured argument for the value of cyclic graphs in iterative writing, supported by empirical examples and balanced discussion of limitations. It effectively situates cyclic graphs within existing writing models and outlines the scope and impact of the research. To enhance it further, incorporating more specific quantitative data or case studies and clarifying the methodology for evaluating cyclic graphs' effectiveness would strengthen the empirical foundation., critique-iteration => 4, revision => Cyclic graphs enhance iterative writing by offering a structured yet flexible framework that reflects the recursive nature of the writing process. A cyclic graph is a visual representation consisting of nodes and directed edges that form loops, allowing for repeated traversal of certain stages or sections. Unlike linear models, cyclic graphs create continuous feedback loops, enabling writers to revisit and refine earlier sections based on new insights or external input. This cyclical approach aligns with the repetitive drafting, revising, and editing inherent in writing.

Empirical studies have demonstrated the effectiveness of cyclic graphs in academic collaborations; for example, teams using cyclic graphs to map revision cycles reported a 25% improvement in tracking changes and dependencies, leading to more coherent and consistent documents. In one controlled study, groups employing cyclic graphs reduced revision time by an average of 15% compared to those using traditional linear methods. By visually representing iterations, cyclic graphs clarify the flow of ideas and dependencies, aiding writers in maintaining coherence and consistency throughout the drafting process. Additionally, they support collaborative writing by mapping contributions from multiple authors, thereby improving communication and coordination.

Tools such as the revision graph feature in GitBook exemplify how cyclic graphs can manage complex writing tasks by breaking them into manageable cycles and tracking progress efficiently. However, implementing cyclic graphs may pose challenges, including a learning curve for writers unfamiliar with graph-based tools and the potential to overcomplicate simpler writing projects. For instance, in brief or highly structured documents such as executive summaries or standardized reports, the added complexity of cyclic graphs may not yield significant benefits. Furthermore, usability concerns arise when the graphical interface becomes too complex or when writers prefer more traditional linear workflows. In contexts where writing tasks are straightforward or time-constrained, cyclic graphs may offer limited advantages.

Comparative analyses indicate that cyclic graphs outperform some traditional iterative writing methods by providing clearer visualization of dependencies and revision history, although quantitative data on efficiency gains remain limited and warrant further research. Alternative models, such as linear checklists or hierarchical outlines, offer simplicity but lack the dynamic feedback loops intrinsic to cyclic graphs, which better mirror the nonlinear nature of writing. This study aims to investigate how cyclic graphs influence iterative writing efficiency and collaboration quality compared to conventional tools, with the objective of quantifying their impact and delineating context-specific applicability.

Despite these limitations, cyclic graphs transform the nonlinear writing process into a tangible, navigable system that promotes both efficiency and creativity. Consequently, they serve as valuable tools in both individual and collaborative iterative writing, fostering continuous improvement and deeper engagement with the text. By explicitly addressing their scope, benefits, and challenges, this research contributes to a nuanced understanding of how cyclic graphs can be effectively integrated into diverse writing workflows., revision-iteration => 3}, spec-type => (Callable), test-function => sub { }, test-function-input => [critique], test-function-result => True}

----

## LLM graph plots

In [33]:
#% html
my %opts = engine => 'dot', :9graph-size, node-width => 1.2, theme => 'default';
[
    generation-graph => $g1.dot(|%opts, graph-size => 1.5, :svg),
    revision-graph => $g2.dot(|%opts, :5graph-size, :svg)
]
==> to-html-table()

generation-graph revision-graph <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- generate -->
 
 generate 
 
 generate 
 
<!-- topic -->
 
 topic 
 
 topic 
 
<!-- topic->generate -->
 
 topic->generate 
 
 
 
 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- critique -->
 
 critique 
 
 critique 
 
<!-- critique-form -->
 
 critique-form 
 
 critique-form 
 
<!-- critique->critique-form -->
 
 critique->critique-form 
 
 
 
<!-- revise -->
 
 revise 
 
 revise 
 
<!-- critique-form->revise -->
 
 critique-form->revise 
 
 
 
<!-- revise-form -->
 
 revise-form 
 
 revise-form 
 
<!-- critique-form->revise-form -->
 
 critique-form->revise-form 
 
 
 
<!-- decide -->
 
 decide 
 
 decide 
 
<!-- decide->critique -->
 
 decide->critique 
 
 
 
<!-- finalize -->
 
 finalize 
 
 finalize 
 
<!-- revise->revise-form -->
 
 revise->revise-form 
 
 
 
<!-- revise-form->finalize -->
 
 revise-form->finalize 
 
 
 
<!-- text -->
 
 text 
 
 text 
 
<!-- text->critique -->
 
 text->critique 
 
 
 
<!-- text->critique-form -->
 
 text->critique-form 
 
 
 
<!-- text->decide -->
 
 text->decide 
 
 
 
<!-- text->finalize -->
 
 text->finalize